In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
print("imported")
import random
import os
import sys
import itertools
sys.path.append('src/')
import nn
import process_data
import nibabel as nib
# import cv2

from __future__ import division, print_function, absolute_import
from sklearn.metrics import confusion_matrix
import scipy.sparse
from scipy.misc import imrotate, imresize
from scipy.ndimage.filters import gaussian_filter
from scipy.ndimage import rotate
from skimage import exposure
from skimage.io import imread, imsave
os.environ["CUDA_VISIBLE_DEVICES"]="0"

from tensorflow.python.client import device_lib
local_device_protos = device_lib.list_local_devices()
print(local_device_protos)

imported


/home/jessica/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 16400034169567473662
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 11573238170
locality {
  bus_id: 1
}
incarnation: 4981897453615498359
physical_device_desc: "device: 0, name: TITAN Xp, pci bus id: 0000:02:00.0"
]


# Modeling Data Setup

### Load Data from File

In [2]:
def one_hot_encode(L, class_labels):
    """
    2D array (image) of segmentation labels -> .npy
    # One Hot Encode the label 2d array -> .npy files with dim (h, w, len(class_labels))
    # num classes will be 8? but currently dynamically allocated based on num colors in all scans.
    """
    h, w = L.shape  # Should be 482, 395 (unless resized)
    try:
        encoded = np.array([list(map(class_labels.index, L.flatten()))])

        L = encoded.reshape(h, w)

        Lhot = np.zeros((L.shape[0], L.shape[1], len(class_labels)))
        for i in range(L.shape[0]):
            for j in range(L.shape[1]):
                Lhot[i,j,L[i,j]] = 1
        return Lhot  # Should be shape (482, 395, 9)
    except Exception as e:
            print(e)

def uncode_one_hot(npy_file):
    """
    .npy file -> JPEG
    """
    pass

def show_images(images, cols = 1, titles = None):
    """Display a list of images in a single figure with matplotlib.
    
    Parameters
    ---------
    images: List of np.arrays compatible with plt.imshow.
    
    cols (Default = 1): Number of columns in figure (number of rows is 
                        set to np.ceil(n_images/float(cols))).
    
    titles: List of titles corresponding to each image. Must have
            the same length as titles.
    """
    assert((titles is None)or (len(images) == len(titles)))
    n_images = len(images)
    if titles is None: titles = ['Image (%d)' % i for i in range(1,n_images + 1)]
    fig = plt.figure()
    for n, (image, title) in enumerate(zip(images, titles)):
        a = fig.add_subplot(cols, np.ceil(n_images/float(cols)), n + 1)
#         if image.ndim == 2:
#             plt.gray()
        plt.imshow(image)
        a.set_title(title)
    fig.set_size_inches(np.array(fig.get_size_inches()) * n_images)
    plt.show()
    
def load_sparse_csr(filename):
    # Sparse matrix reading function to read our raw .npz files
    assert filename.endswith('.npz')
    loader = np.load(filename)  # filename must end with .npz
    return scipy.sparse.csr_matrix((loader['data'], loader['indices'], loader['indptr']),
                      shape=loader['shape'])

def get_raw_pixel_classes():
    #import nibabel as nib
    base_data_dir = "/home/jessica/Documents/hart-seg-ml/allrawnifti"
    example_segmentation = os.path.join(base_data_dir, 'trial8_30_fs_seg_ak5_lh4_TRANS.nii')
    scan_voxel = nib.load(example_segmentation)
    struct_arr = scan_voxel.get_data()
    n, h, w = struct_arr.shape
    class_labels = list(np.unique(struct_arr))
    
def check_one_hot(encoded_img):
    print(encoded_img.shape)
    return np.all(np.sum(encoded_img, axis=2) == 1.)

def batch_img_resize(images, h = 256, w = 256):
    images_resized = np.zeros([0, newHeight, newWidth], dtype=np.uint8)
    for  image in range(images.shape[0]):
        temp = imresize(images[image], [h, w], 'nearest')
        images_resized = np.append(images_resized, np.expand_dims(temp, axis=0), axis=0)
    return images_resized

In [3]:
raw_pixel_classes =[0, 100]  # Expected raw grayscale values for each pixel
#directory = "/Users/nozik/Documents/HARTresearch/allpreprocessed"
directory = "/home/jessica/Documents/hart-seg-ml/fcn_test/artifacts_preprocessed"
filenames = []  # Stores all filenames
raw_images = []  # Stores X (Raw cross section images as 2D np.ndarray)
segmentations = []  # Stores Y (Labeled/Segmented image as one-hot-encoded NumClasses-D np.ndarray)
h, w = 512, 512

In [4]:
for folder in os.listdir(directory):
    class_labels = set()
    if not folder.startswith('.'):
        path = os.path.join(directory, folder)
        print(path)
        files = sorted([f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f)) and not f.startswith('.')])
        
        # Class label sanity check
#         for f in files:
#             if 'label' in f:
#                 img = imread(os.path.join(path, f), flatten=True)
#                 class_labels = class_labels.union(np.unique(img))
#         if not class_labels.issubset(raw_pixel_classes):
#             print("Class labels found in labeled images do not match the expected classes for scan {}".format(folder))
#             print("Expected {}".format(raw_pixel_classes))
#             print("Received {}".format(sorted(class_labels)))
#             break
        
        # Sanity image read and show some images in pairs (play with the range inputs)
#         for f in range(0, len(files), 2):
#             label_name = files[f]
#             raw_name = files[f+1]
#             label_img = imread(os.path.join(path, label_name), flatten=True)
#             raw_img = load_sparse_csr(os.path.join(path, raw_name)).toarray()  # Load sparse csr mat img -> to 2D numpy array
#             show_images([label_img, raw_img], titles=[label_name, raw_name])
        
        # Set up Datasets (X, Y) pairs of data ->
        # files are sorted by the name: either '#_label' or '#_raw'
        for f in files:
            print(f, end=' ')
            if 'label' in f:
                img = imread(os.path.join(path, f), flatten=True)
            else:
                img = load_sparse_csr(os.path.join(path, f)).toarray()            
            
#             imresize(seg[:,:,1],(h,w), interp='nearest')/255.0
            npad = ((15, 15), (58, 59))  # Pads to size 512, 512
            img = np.pad(img, pad_width=npad, mode='constant', constant_values=0)
            if 'raw' in f:
                raw_images.append(img)
            elif 'label' in f:
                encoded_img = one_hot_encode(img, raw_pixel_classes)
                segmentations.append(encoded_img)
            filenames.append(os.path.join(folder, f))
    print("")
            

# print(filenames)

            
# images = np.array(images)
# segmentations = np.round(np.array(segmentations)).astype('uint8')


# study_num = int(2)
# train_lst = np.load('data/splits/train_lst_' + str(study_num) + '.npy')
# val_lst = np.load('data/splits/val_lst_' + str(study_num) + '.npy')

/home/jessica/Documents/hart-seg-ml/fcn_test/artifacts_preprocessed/trial18_90_fs


00000_label.png 00000_raw.npz 00001_label.png 00001_raw.npz 00002_label.png 00002_raw.npz 00003_label.png 00003_raw.npz 00004_label.png 00004_raw.npz 00005_label.png 00005_raw.npz 00006_label.png 00006_raw.npz 00007_label.png 00007_raw.npz 00008_label.png 00008_raw.npz 00009_label.png 00009_raw.npz 00010_label.png 00010_raw.npz 00011_label.png 00011_raw.npz 00012_label.png 00012_raw.npz 00013_label.png 00013_raw.npz 00014_label.png 00014_raw.npz 00015_label.png 00015_raw.npz 00016_label.png 00016_raw.npz 00017_label.png 00017_raw.npz 00018_label.png 00018_raw.npz 00019_label.png 00019_raw.npz 00020_label.png 00020_raw.npz 00021_label.png 00021_raw.npz 00022_label.png 00022_raw.npz 00023_label.png 00023_raw.npz 00024_label.png 00024_raw.npz 00025_label.png 00025_raw.npz 00026_label.png 00026_raw.npz 00027_label.png 00027_raw.npz 00028_label.png 00028_raw.npz 00029_label.png 00029_raw.npz 00030_label.png 00030_raw.npz 00031_label.png 00031_raw.npz 00032_label.png 00032_raw.npz 00033_labe

00273_raw.npz 00274_label.png 00274_raw.npz 00275_label.png 00275_raw.npz 00276_label.png 00276_raw.npz 00277_label.png 00277_raw.npz 00278_label.png 00278_raw.npz 00279_label.png 00279_raw.npz 00280_label.png 00280_raw.npz 00281_label.png 00281_raw.npz 00282_label.png 00282_raw.npz 00283_label.png 00283_raw.npz 00284_label.png 00284_raw.npz 00285_label.png 00285_raw.npz 00286_label.png 00286_raw.npz 00287_label.png 00287_raw.npz 00288_label.png 00288_raw.npz 00289_label.png 00289_raw.npz 00290_label.png 00290_raw.npz 00291_label.png 00291_raw.npz 00292_label.png 00292_raw.npz 00293_label.png 00293_raw.npz 00294_label.png 00294_raw.npz 00295_label.png 00295_raw.npz 00296_label.png 00296_raw.npz 00297_label.png 00297_raw.npz 00298_label.png 00298_raw.npz 00299_label.png 00299_raw.npz 00300_label.png 00300_raw.npz 00301_label.png 00301_raw.npz 00302_label.png 00302_raw.npz 00303_label.png 00303_raw.npz 00304_label.png 00304_raw.npz 00305_label.png 00305_raw.npz 00306_label.png 00306_raw.

00547_raw.npz 00548_label.png 00548_raw.npz 00549_label.png 00549_raw.npz 00550_label.png 00550_raw.npz 00551_label.png 00551_raw.npz 00552_label.png 00552_raw.npz 00553_label.png 00553_raw.npz 00554_label.png 00554_raw.npz 00555_label.png 00555_raw.npz 00556_label.png 00556_raw.npz 00557_label.png 00557_raw.npz 00558_label.png 00558_raw.npz 00559_label.png 00559_raw.npz 00560_label.png 00560_raw.npz 00561_label.png 00561_raw.npz 00562_label.png 00562_raw.npz 00563_label.png 00563_raw.npz 00564_label.png 00564_raw.npz 00565_label.png 00565_raw.npz 00566_label.png 00566_raw.npz 00567_label.png 00567_raw.npz 00568_label.png 00568_raw.npz 00569_label.png 00569_raw.npz 00570_label.png 00570_raw.npz 00571_label.png 00571_raw.npz 00572_label.png 00572_raw.npz 00573_label.png 00573_raw.npz 00574_label.png 00574_raw.npz 00575_label.png 00575_raw.npz 00576_label.png 00576_raw.npz 00577_label.png 00577_raw.npz 00578_label.png 00578_raw.npz 00579_label.png 00579_raw.npz 00580_label.png 00580_raw.

In [ ]:

# raw_90_deg_training = np.array(raw_images)
# seg_90_deg_training = np.array(segmentations)

### Split into Training, Cross Validation and Test sets

In [ ]:
print(len(raw_images), len(segmentations))

In [ ]:
"""
TODO: Same Scan cannot be used across Train, Validation and Test sets
TODO: Different weight conditions and angles may be used to segment other raw_scans
TODO: Bounding Box, image resizing, padding edges
"""
# raw_images holds our X data
# segmentations holds out Y data
x_train, y_train = [], []
x_val, y_val = [], []
x_test, y_test = [], []

percent_train, percent_val, percent_test = 60, 10, 30
num_train = np.round(len(raw_images) * percent_train/100).astype(np.int)
num_val = np.round(num_train + len(raw_images) * percent_val/100).astype(np.int)
num_test = np.round(num_val + len(raw_images) * percent_test/100).astype(np.int)

print("num_train: ", num_train, "num_val: ", num_val, "num_test: ", num_test)

assert len(raw_images) == len(segmentations)
rand_indices = list(np.random.choice(len(raw_images), len(raw_images), replace=False))

for i in rand_indices[:num_train]:
    x_train.append(raw_images[i])
    y_train.append(segmentations[i])
for j in rand_indices[num_train:num_val]:
    x_val.append(raw_images[j])
    y_val.append(segmentations[j])
for k in rand_indices[num_val:num_test]:
    x_test.append(raw_images[k])
    y_test.append(segmentations[k])

        
x_train = np.array(x_train).reshape((len(x_train), h, w, 1))
x_test = np.array(x_test).reshape((len(x_test), h, w, 1))
x_val = np.array(x_val).reshape((len(x_val), h, w, 1))
y_train = np.array(y_train)
y_val = np.array(y_val)
y_test = np.array(y_test)


print(x_train.shape)
print(x_test.shape)
print(x_val.shape)
print(y_train.shape)
print(y_test.shape)
print(y_val.shape)

# Fix data padding to create square 482 by 482 matrix
# npad = ((0, 0), (15, 15), (58, 59), (0, 0))
# x_train = np.pad(x_train, pad_width=npad, mode='constant', constant_values=0)
# x_test = np.pad(x_test, pad_width=npad, mode='constant', constant_values=0)
# y_train = np.pad(y_train, pad_width=npad, mode='constant', constant_values=0)
# y_test = np.pad(y_test, pad_width=npad, mode='constant', constant_values=0)

# print()
# print(x_train.shape)
# print(x_test.shape)
# print(y_train.shape)
# print(y_test.shape)


#     image = imresize(imread(directory + folder + '/' + folder + '.jpg', flatten = True),(h, w))
#     images.append(image)
#     filenames.append(folder)
#     seg = np.load(directory+folder+'/seg.npy')
#     temp = np.zeros((h,w,1))
#     temp[:,:,1] = imresize(seg[:,:,1],(h,w), interp='nearest')/255.0
#     segmentations.append(temp)

# FCN-8s Architecture


In [ ]:
class FCN(object):
    def __init__(self, mean, weight_decay, learning_rate, label_dim=8, dropout=0.9, h=512, w=512):
        self.NUM_CLASSES = 2 # TODO: HARDCODE THIS
        self.label_dim = label_dim
        self.weight_decay = weight_decay
        self.learning_rate = learning_rate
        self.dropout = dropout

        self.x_train = tf.placeholder(tf.float32, [None, h, w, 1])
        self.y_train = tf.placeholder(tf.float32, [None, h, w, 2])
        self.x_test = tf.placeholder(tf.float32, [None, h, w, 1])
        self.y_test = tf.placeholder(tf.float32, [None, h, w, 2])

        self.output = self.fcn(self.x_train, mean, keep_prob=self.dropout)
        self.loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=self.output, labels=self.y_train))
        self.opt = tf.train.AdamOptimizer(self.learning_rate).minimize(self.loss)
        
        self.pred = self.fcn(self.x_test, mean, reuse=True, keep_prob=.8)
        self.loss_summary = tf.summary.scalar('loss', self.loss)

    def fcn(self, input, mean, keep_prob = 0.9, reuse = True):
        input = input - mean  # Demean

        pool_ = lambda x: nn.max_pool(x, 2, 2)
        conv_ = lambda x, output_depth, name, padding='SAME', relu=True, filter_size=3: nn.conv(x, filter_size,
                                                                                                output_depth, 1,
                                                                                                self.weight_decay,
                                                                                                name=name,
                                                                                                padding=padding,
                                                                                                relu=relu)
        deconv_ = lambda x, output_depth, name: nn.deconv(x, 2, output_depth, 2, self.weight_decay, name=name)

        # construct fully convolutional layers
        
        print(input.get_shape())
#         with tf.variable_scope("conv1_1") as scope:
        conv_1_1 = conv_(input, 64, 'conv1_1')
        conv_1_2 = conv_(conv_1_1, 64, 'conv1_2')

        pool_1 = pool_(conv_1_2)

        conv_2_1 = conv_(pool_1, 128, 'conv2_1')
        conv_2_2 = conv_(conv_2_1, 128, 'conv2_2')

        pool_2 = pool_(conv_2_2)
        print(pool_2.get_shape())

        conv_3_1 = conv_(pool_2, 256, 'conv3_1')
        conv_3_2 = conv_(conv_3_1, 256, 'conv3_2')

        self.pool_3 = pool_(conv_3_2)
        print(self.pool_3.get_shape())

        conv_4_1 = conv_(self.pool_3, 512, 'conv4_1')
        conv_4_2 = conv_(conv_4_1, 512, 'conv4_2')

        self.pool_4 = pool_(conv_4_2)
        print(self.pool_4.get_shape())

        conv_5_1 = conv_(self.pool_4, 4096, 'conv5_1')
        conv_5_2 = conv_(conv_5_1, 4096, 'conv5_2')

        pool_5 = pool_(conv_5_2)
        print(pool_5.get_shape())
        

        # weight is the equivalent to the kernel creation
        # tf.nn.conv2d parameters: input, filter, strides, padding, 

        w6 = tf.Variable(tf.random_normal([16, 16, 4096, 4096]), name="weight6")
        b6 = tf.Variable(tf.zeros([4096]), name="bias6")
        conv_6 = tf.nn.conv2d(pool_5, w6, [1, 1, 1, 1], padding="SAME")
        b6_layer = tf.nn.bias_add(conv_6, b6)
        relu_6 = tf.nn.relu(conv_6, name="relu6")
        relu_dropout6 = tf.nn.dropout(relu_6, keep_prob=keep_prob)

        w7 = tf.Variable(tf.random_normal([1, 1, 4096, 4096]), name="weight7")
        b7 = tf.Variable(tf.zeros([4096]), name="bias7")
        conv7 = tf.nn.conv2d(relu_dropout6, w7, [1, 1, 1, 1], padding="SAME") 
        b7_layer = tf.nn.bias_add(conv7, b7)
        relu7 = tf.nn.relu(conv7, name="relu7")
        relu_dropout7 = tf.nn.dropout(relu7, keep_prob=keep_prob)
        print(relu_dropout7.get_shape())
        
        
        # begin decoder, start with 1x1 convolution on conv7
#         w8 = tf.Variable(tf.random_normal([1, 1, 4096, self.NUM_CLASSES]),
#                                    name="W8")
#         b8 = tf.Variable(tf.zeros([self.NUM_CLASSES]), name="b8")
#         conv8 = tf.nn.conv2d(relu_dropout7, w8, [1, 1, 1, 1], padding="SAME")  
#         b8_layer = tf.nn.bias_add(conv8, b8)
        b8_layer = tf.layers.conv2d(relu_dropout7, self.NUM_CLASSES, kernel_size=1, strides=(1, 1), padding="SAME", name="b8_layer")
        print(b8_layer.get_shape())
        # skip layer 1: add compressed conv7 layer to pool4
        
        up1 = tf.layers.conv2d_transpose(b8_layer, 512, kernel_size=4, strides=2, name="up1", padding="SAME")
        print(up1.get_shape())
        print(self.pool_4.get_shape())
        fuse1 = tf.add(up1, self.pool_4)
        
        up2 = tf.layers.conv2d_transpose(fuse1, 256, kernel_size=4, strides=2, name="up2", padding="SAME")
        fuse2 = tf.add(up2, self.pool_3)

        output = tf.layers.conv2d_transpose(fuse2, self.NUM_CLASSES, kernel_size=16, strides=8, name="output", padding="SAME")
        return output
    
    
#         deconv_shape1 = pool_4.get_shape()
#         # 4x4 upscaling kernel with stride 
#         W_t1 = tf.Variable(tf.random_normal([4, 4, deconv_shape1[3].value, self.NUM_CLASSES]), name="W_t1")
#         b_t1 = tf.Variable(tf.zeros([deconv_shape1[3].value]), name="bt1")
#         conv_t1 = tf.layers.conv2d_transpose(conv8, W_t1, tf.shape(pool_4), [2, 2, 1, 1])
#         bt1_layer = tf.nn.bias_add(conv_t1, b_t1)
#         fuse_1 = tf.add(conv_t1, pool_4, name="fuse_1")
#         print(tf.shape(fuse_1))

        # skip layer 2: add skip layer 1 with pool3
    #         deconv_shape2 = pool_3.get_shape()
#         W_t2 = tf.Variable(tf.random_normal([4, 4, deconv_shape2[3].value, deconv_shape1[3].value]), name="W_t2")
#         b_t2 = tf.Variable(tf.zeros([deconv_shape2[3].value]), name="b_t2")
#         conv_t2 = tf.nn.conv2d_transpose(fuse_1, W_t2, tf.shape(pool_3), [2, 2, 1, 1])
#         bt2_layer = tf.nn.bias_add(conv_t2, b_t2)
#         fuse_2 = tf.add(conv_t2, pool_3, name="fuse_2")
#         print(tf.shape(fuse_2))
#         print(self.label_dim)
#         shape = tf.shape(self.label_dim)
#         print(shape)
#         W_t3 = tf.Variable(tf.random_normal([16, 16, self.NUM_CLASSES, deconv_shape2[3].value]), name="W_t3")
#         b_t3 = tf.Variable(tf.zeros([self.NUM_CLASSES]), name="b_t3")

#         prob = tf.nn.conv2d_transpose(fuse_2, W_t3, [1, h, w, self.NUM_CLASSES], [8, 8, 1, 1])
#         prob_bias = tf.nn.bias_add(prob, b_t3)
#         pred = tf.argmax(prob, dimension=3, name="prediction")

In [ ]:


# Hyperparameters
mean = 0
weight_decay = 1e-6
learning_rate = 1e-4
label_dim = 8
maxout = False

# Create TF graph and initialize variables
tf.reset_default_graph()
sess = tf.Session()
model = FCN(mean, weight_decay, learning_rate, label_dim , dropout = 0.9)
sess.run(tf.global_variables_initializer())

In [ ]:
# Restore old model
# saver = tf.train.Saver()
# saver.restore(sess, '/media/deoraid03/jeff/models/a4c_experiments/deep_256_2')

In [ ]:
# Train Model
nn.train(sess, model, x_train, y_train, x_val, y_val, epochs = 1000, batch_size = 1)

In [ ]:
import time

In [ ]:
start = time.time()
# IOU Accuracies for each label
print(nn.validate(sess, model, x_val, y_val))
end = time.time()
print(end - start)

In [ ]:
# Save model
models_dir = '/home/jessica/Documents/hart-seg-ml/artifacts_raw_and_results'
model_name = 'model'
saver = tf.train.Saver()
saver.save(sess, os.path.join(os.path.join(models_dir, model_name), model_name))

In [ ]:
# Load model
saver = tf.train.import_meta_graph('/home/jessica/Documents/hart-seg-ml/models/test_june18/test_june18.meta')
saver.restore(sess, tf.train.latest_checkpoint('./models/test_june18'))

In [ ]:
directory = "/home/jessica/Documents/hart-seg-ml/test"
filenames = []  # Stores all filenames
raw_images = []  # Stores X (Raw cross section images as 2D np.ndarray)
segmentations = []  # Stores Y (Labeled/Segmented image as one-hot-encoded NumClasses-D np.ndarray)

for folder in os.listdir(directory):
    if not folder.startswith('.'):
        path = os.path.join(directory, folder)
        files = sorted([f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f)) and not f.startswith('.')])
        
        for f in files:
            print(f, end=' ')
            if 'label' in f:
                img = imread(os.path.join(path, f), flatten=False)
            else:
                img = load_sparse_csr(os.path.join(path, f)).toarray()            
            
#             imresize(seg[:,:,1],(h,w), interp='nearest')/255.0
            npad = ((15, 15), (58, 59))  # Pads to size 512, 512
            img = np.pad(img, pad_width=npad, mode='constant', constant_values=0)
            if 'raw' in f:
                raw_images.append(img)
            elif 'label' in f:
                # encoded_img = one_hot_encode(img, raw_pixel_classes)
                # segmentations.append(encoded_img)
                segmentations.append(img)
            filenames.append(os.path.join(folder, f))


    

In [ ]:
raw_imgs_arr = np.array(raw_images)
raw_imgs_arr = np.expand_dims(raw_imgs_arr, axis=3)

seg_imgs_arr = np.array(segmentations)
#seg_imgs_arr = np.expand_dims(seg_imgs_arr, axis=3)



print(raw_imgs_arr.shape)

print(seg_imgs_arr.shape)


In [ ]:
def predict_cross_sec(x, model, sess):
    prediction = model.predict(sess, x)
    pred_classes = np.argmax(prediction[0], axis=2)
    return pred_classes

    
def predict_whole_seg(X, model, sess):
    '''
    Todo: Crop the predictions. 
    '''
    # This shape is hardcoded based on size of subject 1's scans.
    #segmented = np.empty((X.shape[0], 482, 395))
    segmented = np.empty(X.shape[:3])
    print("shape: ", segmented.shape)
    num_sections = X.shape[0]
    for i in range(num_sections):
        pred = predict_cross_sec(X[i:i+1], model, sess)
        # segmented[i] = crop_cross_sec(pred)
        segmented[i] = pred
        print(i, end=', ')
    return segmented


def crop_cross_sec(cross_sec):
    '''
    512, 512 -> 482, 395
    npad = ((15, 15), (58, 59))  # Pads to size 512, 512
    img = np.pad(img, pad_width=npad, mode='constant', constant_values=0)
    
    Hardcoded right now. Todo: remember orginal dimensions and parameterize.
    Currently all available scans have the same dimensions, so will work hardcoded for all current scans
    as of 06/13/18
    '''
    return cross_sec[15:512-15,58:512-59]
    

def convert_seg_to_nifti(seg):
    '''
    Hardcoded right now. Todo: generalize for any scan.
    '''
    base_data_dir = "/home/jessica/Documents/hart-seg-ml/allrawnifti"
    original_vol = nib.load(os.path.join(base_data_dir, 'trial15_60_w1_volume_TRANS.nii'))
    new_header = original_vol.header.copy()
    new_nifti = nib.nifti1.Nifti1Image(seg, None, header=new_header)
    save_dir = "/home/jessica/Documents/hart-seg-ml/predictedsegs/u-net_v1.0/30_deg_training"
    save_name = "trial15_60_w1_pred_seg.nii"
    nib.save(new_nifti, os.path.join(save_dir, save_name))
    
def convert_arr_to_nifti(arrs, orig_nii_dir, trial_name, save_dir, save_name, segmented=False):
    '''
    arr should be tuple of numpy arrays of shape (N, height, width). Vol first then seg.
    '''
    orig_nii_files = {}
    
    for file_name in os.listdir(orig_nifti_dir):
        if trial_name in file_name:
            if 'volume' in file_name:
                orig_nii_files['volume'] = file_name
            elif 'seg' in file_name:
                orig_nii_files['seg'] = file_name
    
    print(orig_nii_files)
    
    orig_vol_path = os.path.join(orig_nii_dir, orig_nii_files['volume'])
    orig_seg_path = os.path.join(orig_nii_dir, orig_nii_files['seg'])
    
    
    orig_vol_nii = nib.load(orig_vol_path)
    header = orig_vol_nii.header.copy()
    new_vol_nii = nib.nifti1.Nifti1Image(arrs[0], None, header=header)
    save_name = trial_name + "_proc_filled_volume.nii"
    nib.save(new_vol_nii, os.path.join(save_dir, save_name))
    
    orig_seg_nii = nib.load(orig_seg_path)
    header = orig_seg_nii.header.copy()
    new_seg_nii = nib.nifti1.Nifti1Image(arrs[1], None, header=header)
    save_name = trial_name + "_proc_filled_seg.nii"
    nib.save(new_seg_nii, os.path.join(save_dir, save_name))
    
#     for nii in orig_nii_files:
#         if 'volume' in nii and not segmented:
#             nii_path = os.path.join(orig_nifti_dir, nii)
#         elif 'seg' in nii and segmented:
#             nii_path = os.path.join(orig_nifti_dir, nii)
            
#     print(nii_path)
    
#     orig_nii = nib.load(nii_path)
#     header = orig_nii.header.copy()
#     new_nii = nib.nifti1.Nifti1Image(arr, None, header=header)
    
#     nib.save(new_nii, os.path.join(save_dir, save_name))


In [ ]:
print(raw_imgs_arr.shape)
all_segs = predict_whole_seg(raw_imgs_arr, model, sess)


In [ ]:
print(all_segs.shape)

In [ ]:
convert_seg_to_nifti(all_segs)

In [ ]:
new_shape = raw_imgs_arr.shape[:3]
convert_seg_to_nifti(raw_imgs_arr.reshape(new_shape))

In [ ]:
data_arrs = (raw_imgs_arr, seg_imgs_arr)

orig_nifti_dir = "/home/jessica/Documents/hart-seg-ml/allrawnifti"
curr_trial = "trial20_90_w1"
save_dir = "/home/jessica/Documents/hart-seg-ml/allrawfillednifti"
# save_name = "trial8_30_fs_proc_filled_vol.nii"
save_name = None
convert_arr_to_nifti(data_arrs, orig_nifti_dir, curr_trial, save_dir, save_name)